# 1、ChatMessageHistory的使用

场景1：记忆存储

In [1]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("你好，介绍一下你自己。")
history.add_ai_message("你好！我是一个由先进的人工智能技术驱动的AI助手。我可以帮助你回答问题、提供信息、进行对话，甚至协助完成各种任务。无论你是想了解某个话题、需要建议，还是只是想找个人聊天，我都很乐意为你服务。请告诉我你需要什么帮助吧！")

print(history.messages)

[HumanMessage(content='你好，介绍一下你自己。', additional_kwargs={}, response_metadata={}), AIMessage(content='你好！我是一个由先进的人工智能技术驱动的AI助手。我可以帮助你回答问题、提供信息、进行对话，甚至协助完成各种任务。无论你是想了解某个话题、需要建议，还是只是想找个人聊天，我都很乐意为你服务。请告诉我你需要什么帮助吧！', additional_kwargs={}, response_metadata={})]


场景2：对接大模型

In [2]:
import os
import dotenv
from langchain_community.chat_models import ChatTongyi

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

history = ChatMessageHistory()

history.add_user_message("你好，介绍一下你自己。")
history.add_ai_message("你好！我是一个由先进的人工智能技术驱动的AI助手。我可以帮助你回答问题、提供信息、进行对话，甚至协助完成各种任务。无论你是想了解某个话题、需要建议，还是只是想找个人聊天，我都很乐意为你服务。请告诉我你需要什么帮助吧！")
history.add_user_message("帮我计算 1+1 等于几？")

response = chat_model.invoke(history.messages)
print(response.content)

1 + 1 等于 2。


# 2、ConversationBufferMemory的使用

举例1：

In [4]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

memory.save_context(inputs={"input":"你好，我叫小明"},outputs={"output":"很高兴认识你"})
memory.save_context(inputs={"input":"帮我回答一下1+2*3=？"},outputs={"output":"7"})

print(memory.load_memory_variables({}))

{'history': 'Human: 你好，我叫小明\nAI: 很高兴认识你\nHuman: 帮我回答一下1+2*3=？\nAI: 7'}


C:\Users\yzy\AppData\Local\Temp\ipykernel_185124\4035822917.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


举例2：

In [8]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context(inputs={"input":"你好，我叫小明"},outputs={"output":"很高兴认识你"})
memory.save_context(inputs={"input":"帮我回答一下1+2*3=？"},outputs={"output":"7"})

print(memory.load_memory_variables({}))

print("\n\n")

print(memory.chat_memory.messages)

{'history': [HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='帮我回答一下1+2*3=？', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})]}



[HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='帮我回答一下1+2*3=？', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})]


举例3：结合大模型，提示词模板的使用(PromptTemplate)

In [3]:
import os
import dotenv
from langchain_community.chat_models import ChatTongyi
from langchain_core.prompts import PromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

prompt_template = PromptTemplate.from_template(
    template="""
    你可以与人类对话。
    当前对话历史：{history}
    人类问题：{question}
    回复：
    """
)

memory = ConversationBufferMemory()

chain = LLMChain(
    llm=chat_model,
    prompt=prompt_template,
    memory=memory,
)

response = chain.invoke(input={
    "question": "你好，介绍一下你自己。"
})

print(response)

response = chain.invoke(input={
    "question": "你好，很高兴认识你，我叫小明。"
})
print(response)

response = chain.invoke(input={
    "question": "我叫什么名字呢？"
})
print(response)

{'question': '你好，介绍一下你自己。', 'history': '', 'text': '你好！我是通义千问（Qwen），由通义实验室研发的大语言模型。我可以回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。我支持多种语言，包括但不限于中文、英文、德语、法语、西班牙语等。如果你有任何问题或需要帮助，尽管告诉我！😊'}
{'question': '你好，很高兴认识你，我叫小明。', 'history': 'Human: 你好，介绍一下你自己。\nAI: 你好！我是通义千问（Qwen），由通义实验室研发的大语言模型。我可以回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。我支持多种语言，包括但不限于中文、英文、德语、法语、西班牙语等。如果你有任何问题或需要帮助，尽管告诉我！😊', 'text': '你好，小明！很高兴认识你！😊  \n有什么我可以帮你的吗？或者你想聊点什么？'}
{'question': '我叫什么名字呢？', 'history': 'Human: 你好，介绍一下你自己。\nAI: 你好！我是通义千问（Qwen），由通义实验室研发的大语言模型。我可以回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。我支持多种语言，包括但不限于中文、英文、德语、法语、西班牙语等。如果你有任何问题或需要帮助，尽管告诉我！😊\nHuman: 你好，很高兴认识你，我叫小明。\nAI: 你好，小明！很高兴认识你！😊  \n有什么我可以帮你的吗？或者你想聊点什么？', 'text': '你叫小明！😊  \n很高兴再次和你聊天！'}


举例4：显示的设置memory的key的值

In [2]:
import os
import dotenv
from langchain_community.chat_models import ChatTongyi
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

prompt_template = PromptTemplate.from_template(
    template="""
    你可以与人类对话。
    当前对话历史：{chat_history}
    人类问题：{question}
    回复：
    """
)

memory = ConversationBufferMemory(memory_key="chat_history")

chain = LLMChain(
    llm=chat_model,
    prompt=prompt_template,
    memory=memory,
)

response = chain.invoke(input={
    "question": "你好，介绍一下你自己。"
})

print(response)

response = chain.invoke(input={
    "question": "你好，很高兴认识你，我叫小明。"
})
print(response)

response = chain.invoke(input={
    "question": "我叫什么名字呢？"
})
print(response)

{'question': '你好，介绍一下你自己。', 'chat_history': '', 'text': '你好！我是通义千问（Qwen），由通义实验室研发的超大规模语言模型。我可以回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。我支持多种语言，包括但不限于中文、英文、德语、法语、西班牙语等。\n\n如果你有任何问题或需要帮助，尽管告诉我！😊'}
{'question': '你好，很高兴认识你，我叫小明。', 'chat_history': 'Human: 你好，介绍一下你自己。\nAI: 你好！我是通义千问（Qwen），由通义实验室研发的超大规模语言模型。我可以回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。我支持多种语言，包括但不限于中文、英文、德语、法语、西班牙语等。\n\n如果你有任何问题或需要帮助，尽管告诉我！😊', 'text': '你好，小明！很高兴认识你！😊  \n有什么我可以帮你的吗？或者你想聊点什么？'}
{'question': '我叫什么名字呢？', 'chat_history': 'Human: 你好，介绍一下你自己。\nAI: 你好！我是通义千问（Qwen），由通义实验室研发的超大规模语言模型。我可以回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。我支持多种语言，包括但不限于中文、英文、德语、法语、西班牙语等。\n\n如果你有任何问题或需要帮助，尽管告诉我！😊\nHuman: 你好，很高兴认识你，我叫小明。\nAI: 你好，小明！很高兴认识你！😊  \n有什么我可以帮你的吗？或者你想聊点什么？', 'text': '你叫小明！😊  \n很高兴再次和你聊天！'}


举例5：结合大模型、提示词模板额使用（ChatPromptTemplate）

普通的PromptTemplate和ChatPromptTemplate的区别在于，PromptTemplate是在执行后进行存储，而ChatPromptTemplate是在执行前存储用户的输入+执行后存储输出；首次调用时，PromptTemplate没有历史记录，而ChatPromptTemplate会有历史记录。

In [3]:
import os
import dotenv
from langchain_community.chat_models import ChatTongyi
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个与人类对话的机器人。"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "问题：{question}")
])

memory = ConversationBufferMemory(return_messages=True)
llm_chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    memory=memory,
)

response = llm_chain.invoke(input={
    "question": "中国的首都在哪里？"
})
print(response)
response = llm_chain.invoke(input={
    "question": "我刚刚问了什么？"
})
print(response)

{'question': '中国的首都在哪里？', 'history': [HumanMessage(content='中国的首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国的首都是北京。', additional_kwargs={}, response_metadata={})], 'text': '中国的首都是北京。'}
{'question': '我刚刚问了什么？', 'history': [HumanMessage(content='中国的首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国的首都是北京。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我刚刚问了什么？', additional_kwargs={}, response_metadata={}), AIMessage(content='你刚刚问的是：“中国的首都在哪里？”', additional_kwargs={}, response_metadata={})], 'text': '你刚刚问的是：“中国的首都在哪里？”'}


# 3、ConversationChain的使用

举例1：以PromptTemplate的形式使用ConversationChain

In [5]:
import os
import dotenv
from langchain_community.chat_models import ChatTongyi
from langchain_core.prompts import PromptTemplate, MessagesPlaceholder
from langchain.chains.conversation.base import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

prompt_template = PromptTemplate.from_template(
    template="""
    你可以与人类对话。
    当前对话历史：{history}
    人类问题：{input}
    回复：
    """
)

chain = ConversationChain(
    llm=chat_model,
    prompt=prompt_template,
)

response = chain.invoke(input={
    "input": "你好，介绍一下你自己。"
})

print(response)

response = chain.invoke(input={
    "input": "你好，很高兴认识你，我叫小明。"
})
print(response)

response = chain.invoke(input={
    "input": "我叫什么名字呢？"
})
print(response)

{'input': '你好，介绍一下你自己。', 'history': '', 'response': '你好！我是通义千问（Qwen），由通义实验室研发的超大规模语言模型。我可以回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。我支持多种语言，包括但不限于中文、英文、德语、法语、西班牙语等。如果你有任何问题或需要帮助，随时告诉我！😊'}
{'input': '你好，很高兴认识你，我叫小明。', 'history': 'Human: 你好，介绍一下你自己。\nAI: 你好！我是通义千问（Qwen），由通义实验室研发的超大规模语言模型。我可以回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。我支持多种语言，包括但不限于中文、英文、德语、法语、西班牙语等。如果你有任何问题或需要帮助，随时告诉我！😊', 'response': '你好，小明！很高兴认识你！😊  \n有什么我可以帮你的吗？或者你想聊点什么？'}
{'input': '我叫什么名字呢？', 'history': 'Human: 你好，介绍一下你自己。\nAI: 你好！我是通义千问（Qwen），由通义实验室研发的超大规模语言模型。我可以回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。我支持多种语言，包括但不限于中文、英文、德语、法语、西班牙语等。如果你有任何问题或需要帮助，随时告诉我！😊\nHuman: 你好，很高兴认识你，我叫小明。\nAI: 你好，小明！很高兴认识你！😊  \n有什么我可以帮你的吗？或者你想聊点什么？', 'response': '你叫小明！很高兴再次和你聊天，小明！😊'}
